In [7]:
import pandas as pd
import plotly.express as px

columns = [
        'join_id','dt','chain', 'active_secs_per_day'
        ,'num_l1_txs_inbox','num_l1_txs_output','calldata_bytes_l1_inbox'
        ,'avg_l1_gas_price_on_l1_inbox','avg_l1_gas_price_on_l1_output','avg_l1_gas_price_on_l2'
        ,'l1_gas_used_inbox','l1_gas_used_output','l1_gas_used_combined'
        ,'l1_eth_fees_inbox','l1_eth_fees_output','l1_eth_fees_combined'
        ,'l1_contrib_l2_eth_fees_per_day','l2_contrib_l2_eth_fees_per_day'
        ,'l2_num_txs_per_day','l1_gas_used_on_l2','calldata_bytes_l2_per_day','l2_gas_used','l2_eth_fees_per_day'
        ]

In [8]:
#Unify datasets
dunedf = pd.read_csv('csv_inputs/op_chain_gas_economics_dune_query_2453515_dt2023_09_07.csv') #https://dune.com/queries/2453515 --last 90 days
goldskydf = pd.read_csv('csv_inputs/op_chain_gas_economics_goldsky_zora_pgn_dt2023_09_07.csv') #requires auth - https://dash.goldsky.com/question/9-op-chains-activity-by-period

In [9]:
# Map Chain Names
chain_mappings = {
    'zora': 'Zora Network',
    'pgn': 'Public Goods Network',
    # Add more mappings as needed
}
goldskydf['chain'] = goldskydf['chain'].replace(chain_mappings)

In [10]:
#Configure
#rename cols
dunedf = dunedf.rename(columns={
                        'name':'chain'
                        })
#Generate Join ID Column
dunedf['join_id'] = dunedf['dt'].astype(str).str[:10].str.cat(dunedf['chain'].astype(str).str.lower())
goldskydf['join_id'] = goldskydf['dt'].astype(str).str[:10].str.cat(goldskydf['chain'].astype(str).str.lower())

# display(dunedf.head(10))
# print(dunedf.columns)
# print('---')
# display(goldskydf.head(10))
# print(goldskydf.columns)

combo_df = dunedf.merge(goldskydf, on='join_id', how='left')

# display(combo_df)

for c in columns:
        try:
                combo_df[c] = combo_df[c+'_x'].combine_first(combo_df[c+'_y']) #pick first non-null
        except:
                combo_df[c] = combo_df[c] #nada

combo_df['dt'] = pd.to_datetime(combo_df['dt'])
combo_df = combo_df[columns]

combo_df['gas_compression_ratio'] = combo_df['l1_gas_used_inbox'] / combo_df['l1_gas_used_on_l2']
combo_df['bytes_compression_ratio'] = combo_df['calldata_bytes_l1_inbox'] / combo_df['calldata_bytes_l2_per_day']
combo_df['dt_rank'] = combo_df['dt'].rank(method='dense', ascending=False).astype('int')

In [11]:
display( combo_df.sample(5) )

combo_df.to_csv('outputs/op_chain_gas_economics_sample.csv')

,join_id,dt,chain,active_secs_per_day,num_l1_txs_inbox,num_l1_txs_output,calldata_bytes_l1_inbox,avg_l1_gas_price_on_l1_inbox,avg_l1_gas_price_on_l1_output,avg_l1_gas_price_on_l2,...,l1_contrib_l2_eth_fees_per_day,l2_contrib_l2_eth_fees_per_day,l2_num_txs_per_day,l1_gas_used_on_l2,calldata_bytes_l2_per_day,l2_gas_used,l2_eth_fees_per_day,gas_compression_ratio,bytes_compression_ratio,dt_rank
162,2023-07-28zora network,2023-07-28 00:00:00+00:00,Zora Network,86400.0,145,24,4095210.0,34.521439,34.009968,6.627833e+09,...,1.405807,3.009096,15667.0,6.821384e+07,20933837.0,4.970556e+10,4.414903,1.002717,0.195626,41
137,2023-08-03base,2023-08-03 00:00:00+00:00,Base,86400.0,1508,24,58870456.0,28.770538,20.001096,2.937693e+01,...,36.257562,44.103869,458089.0,1.804413e+09,104678951.0,5.233179e+10,80.361431,0.537877,0.562391,35
84,2023-08-16op mainnet,2023-08-16 00:00:00+00:00,OP Mainnet,86400.0,1336,24,154133869.0,27.135621,28.443839,2.695180e+01,...,71.672966,14.632413,611121.0,3.887868e+09,337053611.0,1.831551e+11,86.305379,0.639615,0.457298,22
116,2023-08-08op mainnet,2023-08-08 00:00:00+00:00,OP Mainnet,86400.0,1594,24,187088490.0,23.122305,24.298956,2.310464e+01,...,73.638392,20.464329,824269.0,4.659603e+09,376881201.0,1.995162e+11,94.102721,0.647632,0.496412,30
299,2023-06-18base,2023-06-18 00:00:00+00:00,Base,86400.0,1432,24,2046297.0,15.291874,15.056151,NaN,...,0.000000,0.000000,0.0,NaN,11232000.0,2.168383e+09,0.000000,NaN,0.182185,81


In [12]:
bl_gas_ratio = combo_df['l1_gas_used_inbox'].sum() / combo_df['l1_gas_used_on_l2'].sum()
bl_bytes_ratio = combo_df['calldata_bytes_l1_inbox'].sum() / combo_df['calldata_bytes_l2_per_day'].sum()
print('Blended gas compression ratio: ' + str(round(bl_gas_ratio*100,1)) + '%')
print('Blended bytes compression ratio: ' + str(round(bl_bytes_ratio*100,1)) + '%')

Blended gas compression ratio: 63.0%
Blended bytes compression ratio: 44.0%


In [ ]:
# Create the scatter plot
combo_df_plot = combo_df[
                        (~combo_df['gas_compression_ratio'].isna()) 
                         & (combo_df['gas_compression_ratio']<1) 
                         & (combo_df['active_secs_per_day'] == 86400)
                         ]

# combo_df_plot = combo_df[(~combo_df['bytes_compression_ratio'].isna())
#                          & (combo_df['active_secs_per_day'] == 86400)]

display(combo_df_plot)

fig = px.scatter(combo_df_plot,
                 x='l1_gas_used_on_l2',
                 y='l1_gas_used_inbox',
                 size = 'gas_compression_ratio',
                 color='chain',
                 title='L1 Gas Used on each Layer by chain')

# Display the plot
fig.show()

##############

fig_bytes = px.scatter(combo_df_plot,
                 x='calldata_bytes_l2_per_day',
                 y='calldata_bytes_l1_inbox',
                 size = 'bytes_compression_ratio',
                 color='chain',
                 title='Calldata Bytese on each Layer by chain')

fig_bytes.show()

In [ ]:
df = combo_df.copy()

In [ ]:
# GPT Built model
from sklearn.model_selection import train_test_split

# Selecting the features and target variable
features = ['l1_gas_used_on_l2', 'calldata_bytes_l2_per_day', 'l2_num_txs_per_day']
target = 'l1_gas_used_inbox'

# Drop rows with missing values for now
df_cleaned = df.dropna(subset=features + [target])

# Splitting the data into training and testing sets (80% train, 20% test)
X = df_cleaned[features]
y = df_cleaned[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.shape, X_test.shape


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Initialize and train the linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model's performance
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

mae, mse, r2

In [ ]:
# Retrieve the coefficients and intercept of the model
coefficients = model.coef_
intercept = model.intercept_

coefficients, intercept


```
r^2 = .9962

l1_gas_used_inbox =
          0.6543×l1_gas_used_on_l2
        − 0.4929×calldata_bytes_l2_per_day
        − 159.8580×l2_num_txs_per_day
        + 40,193,003.72
```
        

In [ ]:
# test other features

# Identify columns that do not include 'l1' and 'compression_ratio', except for the one already identified
potential_features = [col for col in df.columns if 'l1' not in col and 'compression_ratio' not in col]
potential_features.append('l1_gas_used_on_l2')  # add the already identified feature

# Remove the non-feature columns (like identifiers and target)
non_feature_columns = ['Unnamed: 0', 'join_id', 'dt', 'chain']
potential_features = [col for col in potential_features if col not in non_feature_columns]

potential_features



In [ ]:
# Re-preprocess the data considering all potential features
df_cleaned_all = df.dropna(subset=potential_features + [target])

# Splitting the data again
X_all = df_cleaned_all[potential_features]
y_all = df_cleaned_all[target]

X_train_all, X_test_all, y_train_all, y_test_all = train_test_split(X_all, y_all, test_size=0.2, random_state=42)

# Updated function to evaluate features using the new datasets
def evaluate_features_updated(features_list):
    X_train_subset = X_train_all[features_list]
    X_test_subset = X_test_all[features_list]

    # Train the model
    model = LinearRegression()
    model.fit(X_train_subset, y_train_all)

    # Predict and evaluate
    y_pred = model.predict(X_test_subset)
    r2 = r2_score(y_test_all, y_pred)
    return r2

# Re-evaluate models
original_r2_updated = evaluate_features_updated(['l1_gas_used_on_l2', 'calldata_bytes_l2_per_day', 'l2_num_txs_per_day'])
all_features_r2_updated = evaluate_features_updated(potential_features)

# Removing one feature at a time from the full set
drop_one_r2_updated = {}
for feature in potential_features:
    subset = [f for f in potential_features if f != feature]
    r2 = evaluate_features_updated(subset)
    drop_one_r2_updated[feature] = r2

original_r2_updated, all_features_r2_updated, drop_one_r2_updated
print('this shows that our original model was "best"')

In [ ]:
# Train the best model on the original set of features
best_features = ['l1_gas_used_on_l2', 'calldata_bytes_l2_per_day', 'l2_num_txs_per_day']
X_train_best = X_train_all[best_features]
X_test_best = X_test_all[best_features]

best_model = LinearRegression()
best_model.fit(X_train_best, y_train_all)

# Retrieve the coefficients and intercept of the best model
best_coefficients = best_model.coef_
best_intercept = best_model.intercept_

best_coefficients, best_intercept

import matplotlib.pyplot as plt

# Predict with the best model
y_pred_best = best_model.predict(X_test_best)

# Plotting actual vs predicted values
plt.figure(figsize=(10, 7))
plt.scatter(y_test_all, y_pred_best, alpha=0.7)
plt.plot([y_test_all.min(), y_test_all.max()], [y_test_all.min(), y_test_all.max()], 'k--', lw=2)
plt.xlabel('Actual l1_gas_used_inbox')
plt.ylabel('Predicted l1_gas_used_inbox')
plt.title('Actual vs Predicted l1_gas_used_inbox')
plt.grid(True)
plt.show()


In [ ]:
# Test Polynomial Model

# Calculate residuals
residuals = y_test_all - y_pred_best

# Plotting residuals against predicted values
plt.figure(figsize=(10, 7))
plt.scatter(y_pred_best, residuals, alpha=0.7)
plt.axhline(0, color='k', linestyle='--', lw=2)
plt.xlabel('Predicted l1_gas_used_inbox')
plt.ylabel('Residuals')
plt.title('Residuals vs Predicted l1_gas_used_inbox')
plt.grid(True)
plt.show()


In [ ]:
from sklearn.preprocessing import PolynomialFeatures

# Generate polynomial features
poly_transformer = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly_transformer.fit_transform(X_train_best)
X_test_poly = poly_transformer.transform(X_test_best)

# Train the polynomial regression model
poly_model = LinearRegression()
poly_model.fit(X_train_poly, y_train_all)

# Predict and evaluate
y_pred_poly = poly_model.predict(X_test_poly)
r2_poly = r2_score(y_test_all, y_pred_poly)
mae_poly = mean_absolute_error(y_test_all, y_pred_poly)
mse_poly = mean_squared_error(y_test_all, y_pred_poly)

r2_poly, mae_poly, mse_poly


In [ ]:
# Plotting actual vs predicted values for the polynomial regression model
plt.figure(figsize=(10, 7))
plt.scatter(y_test_all, y_pred_poly, alpha=0.7, label='Polynomial Regression')
plt.scatter(y_test_all, y_pred_best, alpha=0.7, color='red', marker='x', label='Linear Regression')
plt.plot([y_test_all.min(), y_test_all.max()], [y_test_all.min(), y_test_all.max()], 'k--', lw=2)
plt.xlabel('Actual l1_gas_used_inbox')
plt.ylabel('Predicted l1_gas_used_inbox')
plt.title('Actual vs Predicted l1_gas_used_inbox (Polynomial vs Linear)')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# Calculate residuals for the polynomial model
residuals_poly = y_test_all - y_pred_poly

# Plotting residuals comparison
plt.figure(figsize=(10, 7))
plt.scatter(y_pred_best, residuals, alpha=0.7, color='red', marker='x', label='Linear Regression Residuals')
plt.scatter(y_pred_poly, residuals_poly, alpha=0.7, color='blue', label='Polynomial Regression Residuals')
plt.axhline(0, color='k', linestyle='--', lw=2)
plt.xlabel('Predicted l1_gas_used_inbox')
plt.ylabel('Residuals')
plt.title('Residuals Comparison (Linear vs Polynomial)')
plt.legend()
plt.grid(True)
plt.show()

# Retrieve the coefficients from the polynomial model
poly_coefficients = poly_model.coef_
poly_intercept = poly_model.intercept_

# Mapping feature names to their polynomial terms
feature_names = poly_transformer.get_feature_names(input_features=best_features)

poly_equation = f"l1_gas_used_inbox = {poly_intercept:.2f}"
for coef, feature in zip(poly_coefficients, feature_names):
    poly_equation += f" + {coef:.2f}*{feature}"

poly_equation


#### Linear Model
$$
\begin{align*}
\text{l1\_gas\_used\_inbox} &= 40,193,003.72 \\
&+ 0.6543 \times \text{l1\_gas\_used\_on\_l2} \\
&- 0.4929 \times \text{calldata\_bytes\_l2\_per\_day} \\
&- 159.8580 \times \text{l2\_num\_txs\_per\_day} \\
R^2 &= 0.9962 \\
\end{align*}


#### Polynomial Model
$$
\begin{align*}
\text{l1\_gas\_used\_inbox} &= -88,399,846.41 \\
&- 1.03 \times \text{l1\_gas\_used\_on\_l2} \\
&+ 13.77 \times \text{calldata\_bytes\_l2\_per\_day} \\
&+ 2824.17 \times \text{l2\_num\_txs\_per\_day} \\
&- 0.00 \times \text{l1\_gas\_used\_on\_l2}^2 \\
&+ 0.00 \times \text{l1\_gas\_used\_on\_l2} \times \text{calldata\_bytes\_l2\_per\_day} \\
&+ 0.00 \times \text{l1\_gas\_used\_on\_l2} \times \text{l2\_num\_txs\_per\_day} \\
&- 0.00 \times \text{calldata\_bytes\_l2\_per\_day}^2 \\
&- 0.00 \times \text{calldata\_bytes\_l2\_per\_day} \times \text{l2\_num\_txs\_per\_day} \\
&- 0.01 \times \text{l2\_num\_txs\_per\_day}^2 \\
R^2 &= 0.9974 \\
\end{align*}


Linear model is much simpler, so let's just use that. It breaks down a bit at larger gas used environments, so we'll refine as we have more days like that.

Note: This only applies to the standard OP Chain config. We have a more detailed projection model for changed configs, but note that the config change model is way more involved and potentially inaccurate